In [36]:
import os, sys
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')



from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import RFE

from numpy import set_printoptions
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB



In [37]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/feature1"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

* kiểm tra và tạo các thư mục (nếu chưa có)
* tập test 33%

# 5. Xây dựng và đánh giá mô hình học sâu Neutual Network

In [38]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from mlxtend.classifier import StackingClassifier
from keras_tuner.tuners import RandomSearch
from scipy.stats import uniform

import numpy as np

In [39]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{exps_dir}/feature1/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 1.0, 1: 1.0}

In [40]:
x_test.shape

(330, 42)

In [41]:
x_train.shape

(1022, 42)

In [42]:
import numpy as np
list(y_train)

['fraud_reported']

In [43]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Dữ liệu label đã được mã hóa
def oneHot(arr):
    labels = np.array(arr)
    
    # Khởi tạo một trình chuyển đổi OneHotEncoder
    encoder = OneHotEncoder(sparse=False)
    
    # Reshape lại dữ liệu labels để có dạng cột (cần là ma trận 2D)
    labels_reshaped = labels.reshape(-1, 1)
    
    # Fit trình chuyển đổi vào dữ liệu
    encoder.fit(labels_reshaped)
    
    # Chuyển đổi labels sang dạng one-hot encoding
    onehot_labels = encoder.transform(labels_reshaped)
    return onehot_labels

y_check=oneHot(y_train)
y_check.shape

(1022, 2)

In [44]:
y_train_onehot=oneHot(y_train)
y_test_onehot=oneHot(y_test)

#### * Xây dựng model:

In [45]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Định nghĩa không gian siêu tham số
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20],
}

# Khởi tạo mô hình Random Forest
rf_model = RandomForestClassifier()

# Sử dụng RandomizedSearchCV để tinh chỉnh siêu tham số
random_search_rf = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=5, scoring='accuracy', cv=5)
random_search_rf.fit(x_train, y_train)

# In ra thông tin của mô hình Random Forest tốt nhất
print("Best Random Forest Hyperparameters:")
print(random_search_rf.best_params_)
n_estimators=random_search_rf.best_params_['n_estimators']
max_depth=random_search_rf.best_params_['max_depth']


Best Random Forest Hyperparameters:
{'n_estimators': 200, 'max_depth': 15}


In [46]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform, randint

# Định nghĩa không gian siêu tham số cho Decision Tree
param_dist_dt = {'criterion': ['gini', 'entropy'],
                 'splitter': ['best', 'random'],
                 'max_depth': randint(3, 10),
                 'min_samples_split': randint(2, 20),
                 'min_samples_leaf': randint(1, 20)}

# Khởi tạo mô hình Decision Tree
dt_model = DecisionTreeClassifier()

# Sử dụng RandomizedSearchCV để tinh chỉnh siêu tham số
random_search_dt = RandomizedSearchCV(dt_model, param_distributions=param_dist_dt, n_iter=5, scoring='accuracy', cv=5)
random_search_dt.fit(x_train, y_train)

# In ra thông tin của mô hình Decision Tree tốt nhất
print("Best Decision Tree Hyperparameters:")
print(random_search_dt.best_params_)
best_dt_model = random_search_dt.best_estimator_
criterion=random_search_dt.best_params_['criterion']
max_depthdt=random_search_dt.best_params_['max_depth']
min_samples_leaf=random_search_dt.best_params_['min_samples_leaf']
splitter=random_search_dt.best_params_['splitter']
min_samples_split=random_search_dt.best_params_['min_samples_split']


Best Decision Tree Hyperparameters:
{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 3, 'min_samples_split': 17, 'splitter': 'best'}


In [97]:
from tensorflow.keras.optimizers.legacy import Adam
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dense, Flatten
from keras.models import Model
from keras.layers import Input
lr=0.0001

def build_model(hp):
    input_shape =(42,1)
    x_input = Input(shape=input_shape)
    print(x_input.shape)

    filters1 = hp.Int('filters1', min_value=32, max_value=512, step=32)
    filters2 = hp.Int('filters2', min_value=32, max_value=512, step=32)
    filters3 = hp.Int('filters3', min_value=32, max_value=512, step=32)
    filters4 = hp.Int('filters4', min_value=32, max_value=512, step=32)
    filters5 = hp.Int('filters5', min_value=32, max_value=512, step=32)
  
    kernel_size = hp.Int('kernel_size', min_value=2, max_value=5, step=1)
    dense_units = hp.Int('dense_units', min_value=128, max_value=4096, step=128)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Block 1
    x = Conv1D(filters1, kernel_size, activation='relu', padding='same', name='block1_conv1')(x_input)
    x = Conv1D(filters1, kernel_size, activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling1D(1, strides=1, name='block1_pool')(x)

    # Block 2
    x = Conv1D(filters2, kernel_size, activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv1D(filters2, kernel_size, activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling1D(1, strides=1, name='block2_pool')(x)

    # Block 3
    x = Conv1D(filters3, kernel_size, activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv1D(filters3, kernel_size, activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv1D(filters3, kernel_size, activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling1D(1, strides=1, name='block3_pool')(x)

    # Block 4
    x = Conv1D(filters4, kernel_size, activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv1D(filters4, kernel_size, activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv1D(filters4, kernel_size, activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling1D(1, strides=1, name='block4_pool')(x)

    # Block 5
    x = Conv1D(filters5, kernel_size, activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv1D(filters5, kernel_size, activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv1D(filters5, kernel_size, activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling1D(1, strides=1, name='block5_pool')(x)

    # x = Flatten(name='flatten')(x)
    # x = Dense(dense_units, activation='relu', name='fc1')(x)
    # x = Dense(dense_units, activation='relu', name='fc2')(x)
    # x = Dense(1, activation='sigmoid', name='predictions')(x)  # Sigmoid for binary classification

    x = GlobalAveragePooling1D()(x)
    #x = GlobalMaxPooling1D()(x)
    x = Dense(2, activation='softmax', name='predictions')(x)  # Sigmoid for binary classification

    # Create model.
    model = Model(x_input, x, name='InceptionV2')
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=30,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=20,
    min_lr=0.000001,
    verbose=1,
)

# Tạo đối tượng RandomSearch tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=12,executions_per_trial=1, directory='VGG16_test')

# Tìm kiếm tham số tốt nhất
tuner.search(x_train.values, y_train_onehot, epochs=600, validation_data=(x_test.values, y_test_onehot),callbacks=[early_stopping,lr_scheduler])

# Lấy mô hình tốt nhất
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Trial 9 Complete [00h 03m 41s]
val_accuracy: 0.7333333492279053

Best val_accuracy So Far: 0.7333333492279053
Total elapsed time: 00h 15m 50s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
320               |160               |filters1
160               |480               |filters2
224               |128               |filters3
160               |320               |filters4
512               |96                |filters5
4                 |2                 |kernel_size
3968              |2816              |dense_units
0.001             |0.001             |learning_rate

(None, 42, 1)
Epoch 1/600
32/32 [==============================] - 7s 193ms/step - loss: 0.6942 - accuracy: 0.4902 - val_loss: 0.6911 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 2/600
32/32 [==============================] - 6s 172ms/step - loss: 0.6933 - accuracy: 0.4863 - val_loss: 0.6928 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 3/600
32/32 [==============================] - 6s 176ms/s

KeyboardInterrupt: 

In [59]:
best_trials = tuner.oracle.get_best_trials(1)[0].hyperparameters.values
best_trials

{'dense_units': 1024,
 'learning_rate': 0.001,
 'filters1': 352,
 'kernel_size': 2,
 'filters2': 192,
 'filters3': 128,
 'filters4': 96,
 'filters5': 96}

In [60]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

(None, 42, 1)
Model: "InceptionV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 42, 1)]           0         
                                                                 
 block1_conv1 (Conv1D)       (None, 42, 352)           1056      
                                                                 
 block1_conv2 (Conv1D)       (None, 42, 352)           248160    
                                                                 
 block1_pool (MaxPooling1D)  (None, 21, 352)           0         
                                                                 
 block2_conv1 (Conv1D)       (None, 21, 192)           135360    
                                                                 
 block2_conv2 (Conv1D)       (None, 21, 192)           73920     
                                                                 
 block2_pool (MaxPooling1D)  (None, 10, 1

In [53]:
predictions = best_model.predict(x_test.values)
predictions = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, predictions)
accuracy

11/11 [==============================] - 0s 7ms/step


0.7666666666666667

In [75]:
stacked_input_expanded = np.expand_dims(stacked_input, axis=-1)
stacked_input_expanded.shape

(330, 2, 1)

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
import numpy as np
def create_keras_model():
    input_shape =(4,1)
    x_input = Input(shape=input_shape)

    filters1 = 288
    filters2 = 448
    filters3 = 32
    filters4 = 224
    filters5 = 192
    kernel_size = 3
    dense_units = 1536
    learning_rate =0.001
    print(x_input.shape)
    # Block 1
    x = Conv1D(filters1, kernel_size, activation='relu', padding='same', name='block1_conv1')(x_input)
    x = Conv1D(filters1, kernel_size, activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling1D(1, strides=1, name='block1_pool')(x)
    print(x.shape,1)
    # Block 2
    x = Conv1D(filters2, kernel_size, activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv1D(filters2, kernel_size, activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling1D(1, strides=1, name='block2_pool')(x)
    print(x.shape,2)

    # Block 3
    x = Conv1D(filters3, kernel_size, activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv1D(filters3, kernel_size, activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv1D(filters3, kernel_size, activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling1D(1, strides=1, name='block3_pool')(x)
    print(x.shape,3)

    # Block 4
    x = Conv1D(filters4, kernel_size, activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv1D(filters4, kernel_size, activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv1D(filters4, kernel_size, activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling1D(1, strides=1, name='block4_pool')(x)
    print(x.shape,4)

    # Block 5
    x = Conv1D(filters5, kernel_size, activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv1D(filters5, kernel_size, activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv1D(filters5, kernel_size, activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling1D(1, strides=1, name='block5_pool')(x)
    print(x.shape,5)

    # x = Flatten(name='flatten')(x)
    # x = Dense(dense_units, activation='relu', name='fc1')(x)
    # x = Dense(dense_units, activation='relu', name='fc2')(x)
    # x = Dense(1, activation='sigmoid', name='predictions')(x)  # Sigmoid for binary classification

    x = GlobalAveragePooling1D()(x)
    #x = GlobalMaxPooling1D()(x)
    x = Dense(2, activation='softmax', name='predictions')(x)  # Sigmoid for binary classification

    # Create model.
    model = Model(x_input, x, name='InceptionV2')
    # Thêm các lớp khác vào đây
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model


# keras_clf = KerasClassifier(build_fn=create_keras_model, epochs=10)
rf_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
dtc_model=DecisionTreeClassifier(criterion=criterion,max_depth=max_depthdt,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split,splitter=splitter)
# Huấn luyện các mô hình cơ sở
rf_model.fit(x_train, y_train)
dtc_model.fit(x_train, y_train)


# Dự đoán đầu ra của các mô hình cơ sở
rf_pred = rf_model.predict(x_test)
dtc_pred = dtc_model.predict(x_test)


# keras_pred = keras_clf.predict(x_test)
stacked_input = np.column_stack((rf_pred,dtc_pred))


#history=model.fit(x=x_train,y=y_train,
          #validation_data=(x_test,y_test),class_weight=class_weights_dict ,
          #batch_size=64,epochs=120, callbacks=[confusion_matrix_callback])

keras_clf = KerasClassifier(build_fn=create_keras_model,batch_size=64,epochs=120,callbacks=[early_stopping,lr_scheduler])

# Clone lại để tránh thay đổi trực tiếp vào mô hình gốc
keras_clf = clone(keras_clf)

# Xây dựng mô hình stacking
stacked_model = StackingClassifier(
    classifiers=[rf_model, dtc_model],
    meta_classifier=keras_clf
)

print(stacked_input.shape,"hello")
# Tiếp tục với việc huấn luyện và đánh giá mô hình stacking
stacked_model.fit(stacked_input, y_test_onehot)
stacked_pred = stacked_model.predict(stacked_input)
stacked_pred

(330, 2) hello
(None, 4, 1)
(None, 4, 288) 1
(None, 4, 448) 2
(None, 4, 32) 3
(None, 4, 224) 4
(None, 4, 192) 5
Epoch 1/120
6/6 [==============================] - 1s 19ms/step - loss: 0.6702 - accuracy: 0.6303 - lr: 0.0010
Epoch 2/120
6/6 [==============================] - 0s 17ms/step - loss: 0.6120 - accuracy: 0.7333 - lr: 0.0010
Epoch 3/120
6/6 [==============================] - 0s 18ms/step - loss: 0.5077 - accuracy: 0.7333 - lr: 0.0010
Epoch 4/120
6/6 [==============================] - 0s 16ms/step - loss: 0.4719 - accuracy: 0.7333 - lr: 0.0010
Epoch 5/120
6/6 [==============================] - 0s 16ms/step - loss: 0.4620 - accuracy: 0.7333 - lr: 0.0010
Epoch 6/120
6/6 [==============================] - 0s 17ms/step - loss: 0.4136 - accuracy: 0.7333 - lr: 0.0010
Epoch 7/120
6/6 [==============================] - 0s 17ms/step - loss: 0.4038 - accuracy: 0.7333 - lr: 0.0010
Epoch 8/120
6/6 [==============================] - 0s 20ms/step - loss: 0.3997 - accuracy: 0.7333 - lr: 0.0010


array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1,

In [78]:
# Dự đoán và đánh giá mô hình stacked
stacked_pred = stacked_model.predict(stacked_input)
stacked_pred_binary = (stacked_pred > 0.5).astype(int)
accuracy_stacked = accuracy_score(y_test_onehot, stacked_pred_binary)
print(f'Accuracy of Stacked Model: {accuracy_stacked}')

6/6 [==============================] - 0s 5ms/step
Accuracy of Stacked Model: 0.8393939393939394


In [79]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Đánh giá độ chính xác
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Hiển thị báo cáo phân loại
print("Classification Report:")
print(classification_report(y_test, predictions,labels=[1,0]))

# # Hiển thị ma trận nhầm lẫn
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred,labels=[1,0]))

Accuracy: 0.7666666666666667
Classification Report:
              precision    recall  f1-score   support

           1       0.60      0.38      0.46        88
           0       0.80      0.91      0.85       242

    accuracy                           0.77       330
   macro avg       0.70      0.64      0.66       330
weighted avg       0.75      0.77      0.75       330



In [80]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix




# Đánh giá độ chính xác
accuracy = accuracy_score(y_test, stacked_pred)
print("Accuracy:", accuracy)

# Hiển thị báo cáo phân loại
print("Classification Report:")
print(classification_report(y_test, stacked_pred,labels=[1,0]))

# # Hiển thị ma trận nhầm lẫn
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred,labels=[1,0]))

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [90]:
def check_result(prediction):
    if isinstance(prediction, np.ndarray):
        if prediction.size == 1:
            prediction = prediction.item()
        else:
            prediction = np.argmax(prediction)  # Lấy chỉ số của giá trị lớn nhất
    elif isinstance(prediction, pd.Series):
        prediction = prediction.item()  # Hoặc sử dụng prediction[0] hoặc prediction.iloc[0]
    if prediction == 1:
        return "Y"
    else: return 'N'

In [91]:
solandung=0
hopdonggianlan=0
for indx in range(len(x_test)):
    y_predict = stacked_model.predict(stacked_input)
    #print(y_predict[0][0])
    Y_check = check_result(y_test_onehot[indx])
    print('True:', Y_check)
    print('Predicted:', check_result(y_predict[indx]))
    print("----------------------------")
    if Y_check==check_result(y_predict[indx]):
        solandung+=1
        if Y_check=='Y':
            hopdonggianlan+=1
    

6/6 [==============================] - 0s 6ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 7ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 7ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 5ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 5ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 5ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 5ms/step
True: N
Predicted: Y
----------------------------
6/6 [==============================] - 0s 6ms/step
True: Y
Predicted: Y
----------------------------
6/6 [==============================] - 0s 5ms/step
True: N
Predicted: N
----------------------------
6/6 [==============================] - 0s 6ms/step
True: N
Predicted: N
-------------------

In [92]:
print(f"Số lần dự đoán đúng: {solandung} chiếm khoảng {(solandung/len(x_test))*100}% tổng lần dự đoán")
print(f"Số hợp đồng gian lận trong lần kiểm tra: {hopdonggianlan} chiếm khoảng {(hopdonggianlan/len(y_test[y_test['fraud_reported']==1]))*100}% so với thực tế")


Số lần dự đoán đúng: 277 chiếm khoảng 83.93939393939394% tổng lần dự đoán
Số hợp đồng gian lận trong lần kiểm tra: 73 chiếm khoảng 82.95454545454545% so với thực tế
